<a href="https://colab.research.google.com/github/saratutuianu/Tornado-detection-using-radar-images/blob/main/tornado_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r ./drive/MyDrive/proiect_ml/requirements/torch.txt

In [ ]:
import sys

sys.path.append('/content/drive/MyDrive/proiect_ml')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib, linecache

import torch
from torch import optim, nn
from torch.utils.data import Dataset
from torchvision import transforms, utils

import tornet.data.preprocess as preprocess

linecache.checkcache(preprocess.__file__)      # invalidează cache-ul de linii
preprocess = importlib.reload(preprocess)

from tornet.data.loader import read_file, TornadoDataLoader
from tornet.data.preprocess import add_coordinates, permute_dims, remove_tilt_dim, get_shape
from tornet.data.constants import ALL_VARIABLES

data_root=r'./drive/MyDrive/proiect_ml/tornet_dataset'

year = 2019

catalog_path = os.path.join(data_root,'catalog.csv')

# interpreteaza ca date si ore
catalog = pd.read_csv(catalog_path,parse_dates=['start_time','end_time'])

catalog['J'] = catalog['start_time'].dt.dayofyear
catalog['r'] = catalog['J'] % 20
catalog = catalog[catalog.start_time.dt.year == 2019]

catalog_test = catalog[catalog['type'] == 'test']
catalog_test = catalog_test.sample(frac=1,random_state=1234) # shuffles list

catalog = catalog[catalog['type']=='train']

catalog_train = catalog[catalog['r'] <= 13]
catalog_train = catalog_train.sample(frac=1,random_state=1234) # shuffles list

catalog_validation = catalog[(catalog['r'] > 13) & (catalog['r'] < 17)]
catalog_validation = catalog_validation.sample(frac=1,random_state=1234) # shuffles list

confirmed_number = len(catalog_train[catalog_train['category'] == 'TOR'])
total_number = len(catalog_train)

class TornadoDataset(TornadoDataLoader,Dataset):
    pass

transform = transforms.Compose([
            lambda d: remove_tilt_dim(d)
            ])

file_list_train = [os.path.join(data_root,f) for f in catalog_train.filename]
file_list_validation = [os.path.join(data_root,f) for f in catalog_validation.filename]
file_list_test = [os.path.join(data_root,f) for f in catalog_test.filename]

torch_ds_train = TornadoDataset(file_list_train,
                          variables=ALL_VARIABLES,
                          n_frames=4,
                          tilt_last=False, # so ordering of dims is [time,tilt,az,range]
                          transform=transform)

torch_ds_validation = TornadoDataset(file_list_validation,
                          variables=ALL_VARIABLES,
                          n_frames=4,
                          tilt_last=False, # so ordering of dims is [time,tilt,az,range]
                          transform=transform)

torch_ds_test = TornadoDataset(file_list_test,
                          variables=ALL_VARIABLES,
                          n_frames=4,
                          tilt_last=False, # so ordering of dims is [time,tilt,az,range]
                          transform=transform)


batch_size=16

torch_dl_train = torch.utils.data.DataLoader( torch_ds_train,
                                        batch_size=batch_size,
                                        num_workers=8 )

torch_dl_validation = torch.utils.data.DataLoader( torch_ds_validation,
                                        batch_size=batch_size,
                                        num_workers=8 )

torch_dl_test = torch.utils.data.DataLoader( torch_ds_test,
                                        batch_size=batch_size,
                                        num_workers=8 )


In [ ]:
from tornet.models.torch.cnn_baseline import NormalizeVariable
from tornet.data.constants import CHANNEL_MIN_MAX

def conv3d_bn_block(in_channels, out_channels, kernel_size=3, stride=1, padding='same'):
    return nn.Sequential(
        nn.Conv3d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
        nn.BatchNorm3d(out_channels),
        nn.ReLU(inplace=True)
    )

def conv3d_transpose_bn_block(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    return nn.Sequential(
        nn.ConvTranspose3d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
    )

class TornadoLikelihood(nn.Module):
    def __init__(self,radar_variables=ALL_VARIABLES):
      super(TornadoLikelihood, self).__init__()
      self.radar_variables=radar_variables

      # Partea de encoder
      self.conv_layer_encoder1 = conv3d_bn_block(in_channels=len(radar_variables), out_channels=16)
      self.conv_layer_encoder2 = conv3d_bn_block(in_channels=16, out_channels=16)

      self.max_pool_encoder1 = nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2))

      self.conv_layer_encoder3 = conv3d_bn_block(in_channels=16, out_channels=32)
      self.conv_layer_encoder4 = conv3d_bn_block(in_channels=32, out_channels=32)

      self.max_pool_encoder2 = nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2))

      self.conv_layer_encoder5 = conv3d_bn_block(in_channels=32, out_channels=64)
      self.conv_layer_encoder6 = conv3d_bn_block(in_channels=64, out_channels=64)

      self.max_pool_encoder3 = nn.MaxPool3d(kernel_size=2, stride=2)

      self.conv_layer_encoder7 = conv3d_bn_block(in_channels=64, out_channels=128)
      self.conv_layer_encoder8 = conv3d_bn_block(in_channels=128, out_channels=128)

      self.max_pool_encoder4 = nn.MaxPool3d(kernel_size=2, stride=2)

      # Partea de decoder
      self.conv_layer_decoder1 = conv3d_bn_block(in_channels=128, out_channels=128)
      self.conv_layer_decoder2 = conv3d_bn_block(in_channels=128, out_channels=128)

      self.upsample_decoder1 = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True)

      self.conv_layer_decoder3 = conv3d_bn_block(in_channels=128, out_channels=64)
      self.conv_layer_decoder4 = conv3d_bn_block(in_channels=64, out_channels=64)

      self.upsample_decoder2 = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True)

      self.conv_layer_decoder5 = conv3d_bn_block(in_channels=64, out_channels=32)
      self.conv_layer_decoder6 = conv3d_bn_block(in_channels=32, out_channels=32)

      self.upsample_decoder3 = nn.Upsample(scale_factor=(1,2,2), mode='trilinear', align_corners=True)

      self.conv_layer_decoder7 = conv3d_bn_block(in_channels=32, out_channels=16)
      self.conv_layer_decoder8 = conv3d_bn_block(in_channels=16, out_channels=16)

      self.upsample_decoder4 = nn.Upsample(scale_factor=(1,2,2), mode='trilinear', align_corners=True)

      self.conv_layer_final = conv3d_transpose_bn_block(in_channels=16, out_channels=1)

    def _normalize_inputs(self,data):
        normed_data = {}
        for v in self.radar_variables:
            min_max = np.array(CHANNEL_MIN_MAX[v]) # [2,]
            scale = 1/(min_max[1]-min_max[0])
            offset = min_max[0]
            normed_data[v] = (data[v] - offset) * scale

        return normed_data

    def forward(self,x):
      """
      Assumes x contains radar varialbes on [batch,tilt,az,rng]
      """
      # extract radar inputs
      x = {v:x[v] for v in self.radar_variables} # each [batch,time,Az,Rng]
      # normalize
      x = self._normalize_inputs(x) # each [batch,time,Az,Rng]
      # concatenate along channel (time) dim
      x = torch.stack([x[v] for v in self.radar_variables], dim=1)

      x = torch.where(torch.isnan(x),-3,x)

      x = self.conv_layer_encoder1(x)
      x = self.conv_layer_encoder2(x)
      x = self.max_pool_encoder1(x)

      x = self.conv_layer_encoder3(x)
      x = self.conv_layer_encoder4(x)
      x = self.max_pool_encoder2(x)

      x = self.conv_layer_encoder5(x)
      x = self.conv_layer_encoder6(x)
      x = self.max_pool_encoder3(x)

      x = self.conv_layer_encoder7(x)
      x = self.conv_layer_encoder8(x)
      x = self.max_pool_encoder4(x)

      x = self.conv_layer_decoder1(x)
      x = self.conv_layer_decoder2(x)
      x = self.upsample_decoder1(x)

      x = self.conv_layer_decoder3(x)
      x = self.conv_layer_decoder4(x)
      x = self.upsample_decoder2(x)

      x = self.conv_layer_decoder5(x)
      x = self.conv_layer_decoder6(x)
      x = self.upsample_decoder3(x)

      x = self.conv_layer_decoder7(x)
      x = self.conv_layer_decoder8(x)
      x = self.upsample_decoder4(x)

      x = self.conv_layer_final(x)

      return x


In [ ]:
import torchmetrics
from torchmetrics.functional.classification import binary_auroc, binary_average_precision

def accuracy(logits, y, tres):
  correct = 0
  for i in range(len(logits)):
    if logits[i] >= tres and y[i] == 1:
      correct += 1
    elif logits[i] < tres and y[i] == 0:
      correct += 1

  return correct / float(len(y)) * 100.0

def recall(logits, y, tres):
  correct = 0
  false_negatives = 0
  for i in range(len(logits)):
    if logits[i] >= tres and y[i] == 1:
      correct += 1
    elif logits[i] < tres and y[i] == 1:
      false_negatives += 1

  return 0.0 if correct + false_negatives == 0 else correct / (correct + false_negatives) * 100.0

def precision(logits, y, tres):
  correct = 0
  false_positives = 0
  for i in range(len(logits)):
    if logits[i] >= tres and y[i] == 1:
      correct += 1
    elif logits[i] >= tres and y[i] == 0:
      false_positives += 1

  return 0.0 if correct + false_positives == 0 else correct / (correct + false_positives) * 100.0

def f1_score(logits, y, tres):
  p = precision(logits, y, tres)
  r = recall(logits, y, tres)
  return 0.0 if p + r == 0 else 2 * (p * r) / (p + r)

def apply_metrics(logits, y, avg_train_loss, avg_val_loss, best_treshold):
  acc = accuracy(logits, y, best_treshold)
  rec = recall(logits, y, best_treshold)
  prec = precision(logits, y, best_treshold)
  f1 = f1_score(logits, y, best_treshold)
  auroc = binary_auroc(logits, y)
  auprc = binary_average_precision(logits, y)

  print(f"Accuracy: , {acc:.4f}")
  print(f"Recall: , {rec:.4f}")
  print(f"Precision: , {prec:.4f}")
  print(f"F1 Score: , {f1:.4f}")
  print(f"AUROC:  {auroc.item():.4f}")
  print(f"AUPRC:  {auprc.item():.4f}")

  return {
      'accuracy': acc,
      'recall': rec,
      'precision': prec,
      'f1_score': f1,
      'auroc': auroc.item(),
      'auprc': auprc.item(),
      'avg_train_loss': avg_train_loss,
      'avg_val_loss': avg_val_loss,
      'threshold': best_treshold
  }



In [ ]:
# recall mai important! -> e mai important sa flaguiasca toate tornadele, decat sa nu emita avertizari negative
!pip install plotnine
import plotnine
from plotnine import *
from IPython.display import display

from sklearn.metrics import precision_recall_curve

def find_best_tres(logits, y, beta):
  precision, recall, thresholds = precision_recall_curve(y, logits)

  # Plot the precision-recall curve
  df_recall_precision = pd.DataFrame({'Precision':precision[:-1],
                                      'Recall':recall[:-1],
                                      'Threshold':thresholds})
  df_recall_precision.head()

  plotnine.options.figure_size = (8, 4.8)

  # Creat a data viz
  plot1 = (
      ggplot(data = df_recall_precision) +
      geom_point(aes(x='Recall', y='Precision'), size=0.4) +
      geom_line(aes(x='Recall', y='Precision')) +
      labs(title='Recall Precision Curve') +
      xlab('Recall') +
      ylab('Precision') +
      theme_minimal()
  )
  display(plot1)

  precision = precision[:-1]
  recall = recall[:-1]

  den = (beta**2)*precision + recall
  fbeta_score = np.where(den > 0, (1 + beta**2) * precision * recall / den, -np.inf)

  # Find the optimal threshold
  index = np.argmax(fbeta_score)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fbeta_score[index], ndigits = 4)
  recallOpt = round(recall[index], ndigits = 4)
  precisionOpt = round(precision[index], ndigits = 4)
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))
  print('Recall: {}, Precision: {}'.format(recallOpt, precisionOpt))

  plotnine.options.figure_size = (8, 4.8)

  # Create a data viz
  plot2 = (
    ggplot(data = df_recall_precision)+
    geom_point(aes(x = 'Recall',
                    y = 'Precision'),
                size = 0.4)+
    # Best threshold
    geom_point(aes(x = recallOpt,
                    y = precisionOpt),
                color = '#981220',
                size = 4)+
    geom_line(aes(x = 'Recall',
                  y = 'Precision'))+
    # Annotate the text
    geom_text(aes(x = recallOpt,
                  y = precisionOpt),
              label = 'Optimal threshold \n for class: {}'.format(thresholdOpt),
              nudge_x = 0.18,
              nudge_y = 0,
              size = 10,
              fontstyle = 'italic')+
    labs(title = 'Recall Precision Curve')+
    xlab('Recall')+
    ylab('Precision')+
    theme_minimal()
  )

  display(plot2)

  return thresholdOpt


In [ ]:
from time import time
import logging

training_logs = []
class TrainingLogger:
    def __init__(self):
        logging.basicConfig(level=logging.INFO, format='%(message)s', filename='/content/drive/MyDrive/proiect_ml/training.log', filemode='a')

    def on_epoch_begin(self, epoch):
        self.epoch_start_time = time()
        logging.info(f"Epoch {epoch + 1} starting.")

    def on_epoch_end(self, epoch, logs=None):
        elapsed_time = time() - self.epoch_start_time
        logging.info(f"Epoch {epoch + 1} finished in {elapsed_time:.2f} seconds.")
        logs['epoch_time'] = elapsed_time  # Add epoch time to logs
        training_logs.append(logs)  # Collect training logs
        logging.info(f"Epoch {epoch + 1}: Val loss = {logs['avg_val_loss']:.4f}, Accuracy = {logs['accuracy']:.4f}, Recall = {logs['recall']:.4f}, Precision = {logs['precision']:.4f}, F1 = {logs['f1_score']:.4f}, AUROC = {logs['auroc']:.4f}, AUPRC = {logs['auprc']:.4f}")


In [ ]:
from torch.optim import Optimizer

class RMSpropFromScratch(Optimizer):
    def __init__(self, params, lr=1e-3, alpha=0.99, eps=1e-8,
                 weight_decay=0.0, momentum=0.0, centered=False):

        defaults = dict(lr=lr, alpha=alpha, eps=eps,
                        weight_decay=weight_decay, momentum=momentum,
                        centered=centered)
        super().__init__(params, defaults)

    @torch.no_grad()
    def step(self, closure=None):
        loss = None

        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            lr = group['lr']
            alpha = group['alpha']
            eps = group['eps']
            wd = group['weight_decay']

            for p in group['params']:
                if p.grad is None:
                    continue

                grad = p.grad

                # weight decay: g <- g + wd * w
                if wd != 0.0:
                    grad = grad.add(p, alpha=wd)

                state = self.state[p]
                # init state
                if len(state) == 0:
                    state["square_avg"] = torch.zeros_like(p, memory_format=torch.preserve_format)  # E[g^2]

                    if group["momentum"] > 0:
                      state["momentum_buffer"] = torch.zeros_like(p, memory_format=torch.preserve_format)

                    if group["centered"]:
                      state["grad_avg"] = torch.zeros_like(p, memory_format=torch.preserve_format)  # E[g]

                square_avg = state['square_avg']
                square_avg.mul_(alpha).addcmul_(grad, grad, value=1 - alpha)

                if group['centered']:
                    grad_avg = state['grad_avg']
                    grad_avg.mul_(alpha).add_(grad, alpha=1 - alpha)
                    # var ≈ E[g^2] - (E[g])^2
                    avg = square_avg.addcmul(grad_avg, grad_avg, value=-1).sqrt_()
                else:
                    avg = square_avg.sqrt()

                avg = avg.add_(eps)

                if group['momentum'] > 0.0:
                    buf = state['momentum_buffer']
                    buf.mul_(['momentum']).addcdiv_(grad, avg)
                    p.add_(buf, alpha=-lr)
                else:
                    p.addcdiv_(grad, avg, value=-lr)

        return loss

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import logging
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from time import time

from typing import Dict, List, Tuple, Any
import numpy as np

from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

num_epochs = 15

best_val = 10

model = TornadoLikelihood()

pos_weight = torch.tensor([(total_number - confirmed_number)/confirmed_number])  # weight asociat cu clasa pozitiva
loss_f = nn.BCEWithLogitsLoss(pos_weight=pos_weight)  # daca w_p > 1 => favorizeaza recall, invers => precision

optimizer = RMSpropFromScratch(model.parameters(), lr=1e-3)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# TensorBoard setup
writer = SummaryWriter('/content/drive/MyDrive/proiect_ml/summary')

ckpt_path = "/content/drive/MyDrive/proiect_ml/checkpoints/best.pth"
ckpt = torch.load(ckpt_path)

model.load_state_dict(ckpt["model_state"])
optimizer.load_state_dict(ckpt["optimizer_state"])
scheduler.load_state_dict(ckpt["scheduler_state"])

start_epoch = int(ckpt["epoch"]) + 1   # <- va fi 1
best_val = float(ckpt.get("val_loss", 1e9))

print(f"Reluare din {ckpt_path} | start_epoch={start_epoch} | best_val={best_val:.4f}")

best_treshold = 0.6

avg_val_loss = best_val
avg_train_loss = 0.0

In [ ]:

patience = 5
patience_counter = 0

callback = TrainingLogger()

# Training loop
total_step = len(torch_dl_train)

eps = 0.05

for epoch in range(start_epoch, num_epochs):
    callback.on_epoch_begin(epoch)
    model.train()
    train_loss = 0.0

    for i, batch in enumerate(torch_dl_train):
        y = torch.squeeze(batch.pop('label'))
        y_float = y[:, -1].float()

        # De ce label smoothing? Nu e niciodata 100% ca e tornada
        y_smooth = y_float * (1 - 2*eps) + eps

        # Forward pass
        logits = model(batch) # [batch,1,T,L,W]
        logits = F.max_pool3d(logits, kernel_size=logits.size()[2:]) # [batch,1,1,1,1]
        logits = torch.squeeze(logits) # [batch,1] for binary classifications
        loss = loss_f(logits, y_smooth)

        # Backward and optimize
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

    # Calculate average training loss for the epoch
    avg_train_loss = train_loss / len(torch_dl_train)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Training Loss: {avg_train_loss:.4f}')
    writer.add_scalar('training loss', avg_train_loss, epoch)

    ckpt_path = f"/content/drive/MyDrive/proiect_ml/checkpoints/epoch_{epoch+1:03d}.pth"
    torch.save({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "scheduler_state": scheduler.state_dict(),
        "val_loss": avg_train_loss,
    }, ckpt_path)

    # Validation
    all_probs = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        val_loss = 0.0
        for i, batch in enumerate(torch_dl_validation):
            y = torch.squeeze(batch.pop('label'))
            y_float = y[:, -1].float()
            y_int = y[:, -1].long()

            # Forward pass
            logits = model(batch) # [batch,1,T,L,W]
            logits = F.max_pool3d(logits, kernel_size=logits.size()[2:]) # [batch,1,1,1,1]
            logits = torch.squeeze(logits) # [batch,1] for binary classifications
            loss = loss_f(logits, y_float)

            all_probs.append(torch.sigmoid(logits))
            all_labels.append(y_int)

            val_loss += loss.item()

        avg_val_loss = val_loss / len(torch_dl_validation)
        writer.add_scalar('validation loss', avg_val_loss, epoch)
        all_probs = torch.cat(all_probs, dim = 0)
        all_labels = torch.cat(all_labels, dim = 0)

    if avg_val_loss < best_val:
        best_val = avg_val_loss
        best_path = "/content/drive/MyDrive/proiect_ml/checkpoints/best.pth"
        torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "scheduler_state": scheduler.state_dict(),
            "val_loss": avg_val_loss,
        }, best_path)
        print(f"[Epoch {epoch+1}] Best model salvat la {best_path} (val_loss={best_val:.4f})")

        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}.")
            break

    # Learning rate scheduling
    scheduler.step(avg_val_loss)

    logs = apply_metrics(all_probs, all_labels, avg_train_loss, avg_val_loss, best_treshold)

    best_treshold = find_best_tres(all_probs, all_labels, 2)

    callback.on_epoch_end(epoch, logs)


In [ ]:
# Final test

all_probs = []
all_labels = []

model.eval()
with torch.no_grad():
  for i, batch in enumerate(torch_dl_test):
        y = torch.squeeze(batch.pop('label'))
        y_float = y[:, -1].float()
        y_int = y[:, -1].long()

        # Forward pass
        logits = model(batch) # [batch,1,T,L,W]
        logits = F.max_pool3d(logits, kernel_size=logits.size()[2:]) # [batch,1,1,1,1]
        logits = torch.squeeze(logits) # [batch,1] for binary classifications

        all_probs.append(torch.sigmoid(logits))
        all_labels.append(y_int)

all_probs = torch.cat(all_probs, dim = 0)
all_labels = torch.cat(all_labels, dim = 0)
logs = apply_metrics(all_probs, all_labels, avg_train_loss, avg_val_loss, best_treshold)

callback.on_epoch_end(epoch, logs)

writer.close()